## Create training data set

Toolkit to transform the grid of stellar atmosphere model inputs to a uniform data set of inputs for training the neural network model.

Firstly import the packages required...

In [ ]:
% pylab inline
import numpy as np

from scipy.interpolate import UnivariateSpline, interp1d
from scipy.integrate import simps

from astropy.io import fits
from astropy.utils.console import ProgressBar

import pandas as pd

import matplotlib.pylab as plt
import seaborn as sb

The steps required

- Build a function for one stellar atmosphere model
    - to convolve with each bandpass
    - integrate over wavelength
    - for each given mu angle
    - correct for radius definition
- Loop through all of the atmosphere models
- Export to file

Firstly, let's define the stellar radius correction as the mu from the atmosphere models with the highest intensity gradient

In [ ]:
def new_get_rmax(mu,I0):
    # convert mu to r
    r = np.sqrt(1.-(mu**2.))
    
    # find the maximum derivative point
    i = np.argmax(abs(np.diff(I0)/np.diff(r)))
    
    # make radius correction to values inside new radius
    r_new  = r[i:]/r[i]
    mu_new = np.sqrt(1-r_new**2)
    
    ip_new = I0[i:]
    return r_new, mu_new, ip_new, r[i], mu[i]

Secondly let's define our table of bandpasses to use for convolving with the model stellar atmospheres

In [ ]:
transmission = {'Kp' : './response_functions/kepler_response.txt',
                'S1' : './response_functions/spitzer_1.txt',
                'S2' : './response_functions/spitzer_2.txt',
                'S3' : './response_functions/spitzer_3.txt',
                'S4' : './response_functions/spitzer_4.txt',
                'u'  : './response_functions/Stroemgren_u.txt',
                'v'  : './response_functions/Stroemgren_v.txt',
                'b'  : './response_functions/Stroemgren_b.txt',
                'y'  : './response_functions/Stroemgren_y.txt',
                'U'  : './response_functions/Bessel_U-1.txt',
                'V'  : './response_functions/Bessel_V-1.txt',
                'B'  : './response_functions/Bessel_B-1.txt',
                'R'  : './response_functions/Bessel_R-1.txt',
                'I'  : './response_functions/Bessel_I-1.txt',
                'J'  : './response_functions/2MASS_Jband.txt',
                'H'  : './response_functions/2MASS_Hband.txt',
                'K'  : './response_functions/2MASS_Kband.txt',
                'u_' : './response_functions/SDSS_u.txt',
                'g_' : './response_functions/SDSS_g.txt',
                'r_' : './response_functions/SDSS_r.txt',
                'i_' : './response_functions/SDSS_i.txt',
                'z_' : './response_functions/SDSS_z.txt',
                'vT' : './response_functions/v_tycho.txt',
                'bT' : './response_functions/b_tycho.txt',
                'Cp' : './response_functions/cheops.txt'
               }
filters = {}
for filt in transmission:
    filters[filt] = np.loadtxt(transmission[filt])


Now the definition of retreiving the required information from the model atmosphere

In [ ]:
def read_PHOENIX(chosen_path):
    with fits.open(chosen_path) as f:
        I = (f[0].data)/100.
        mu = f['MU'].data
        CRVAL1 = f[0].header['CRVAL1']
        CDELT1 = f[0].header['CDELT1']
        teff = f[0].header['PHXTEFF']
        logg = f[0].header['PHXLOGG']
        feh = f[0].header['PHXM_H']
    wavelengths = (np.arange(I.shape[1]) * CDELT1 + CRVAL1)/10. # convert to nm to match response functions
    return wavelengths, I, mu, teff, logg, feh

Locate all the atmosphere models, with the list of paths stored in: `model_list`

In [ ]:
model_list = []
model_list.extend(glob.glob('../phoenix2011/Z-0.0/*.fits'))
model_list.extend(glob.glob('../phoenix2011/Z-0.5/*.fits'))
model_list.extend(glob.glob('../phoenix2011/Z-1.0/*.fits'))
model_list.extend(glob.glob('../phoenix2011/Z-1.5/*.fits'))
model_list.extend(glob.glob('../phoenix2011/Z-2.0/*.fits'))
model_list.extend(glob.glob('../phoenix2011/Z-3.0/*.fits'))
model_list.extend(glob.glob('../phoenix2011/Z-4.0/*.fits'))
model_list.extend(glob.glob('../phoenix2011/Z+0.5/*.fits'))
model_list.extend(glob.glob('../phoenix2011/Z+1.0/*.fits'))
print("Number of models found: ",len(model_list))

Since the calculated wavelengths in each model atmosphere output file are the same, we can save some time by interpolating the bandpasses for each filter to match the PHOENIX model output for the convolution and integration step. The interpolated bandpasses are then stored in the dictionary, `filt_int`.

In [ ]:
with fits.open(model_list[0]) as f:
    CRVAL1 = f[0].header['CRVAL1']
    CDELT1 = f[0].header['CDELT1']
    NAXIS1 = f[0].header['NAXIS1']
initial_wavelengths = ((np.arange(NAXIS1) * CDELT1) + CRVAL1)/10. #convert to nm 


filt_int = {}
for filt in filters:
    filt_int[filt] = np.interp(initial_wavelengths,filters[filt][:,0],filters[filt][:,1])

Now we can produce the grid itself

In [ ]:
columns = ['Teff','logg','Z','Filt','mu','intensity','mu_fac','r_fac']

grid1 = []
grid2 = []

with ProgressBar(len(model_list), ipython_widget=False) as bar:
    for item in model_list:
        wavelengths, I, mu, teff, logg, feh = read_PHOENIX(item)

        for filt in filters:
            filt_spec = (I * filt_int[filt]).T

            flux = []
            for j in range(mu.shape[0]):
                flux.append(simps(filt_spec[:,j],wavelengths))

            flux = flux/(flux[-1])
            flux = np.array(flux)
            
            new_r,new_mu,new_I0,r_fac,mu_fac = new_get_rmax(mu,flux)
            
            even_mus = np.linspace(new_mu.min(),1,200)
            interp_I = interp1d(new_mu,new_I0,kind='quadratic',assume_sorted=True)(even_mus)
            
            for q in range(mu.shape[0]):
                grid1.append([teff,logg,feh,filt,mu[q],flux[q],mu_fac,r_fac])

            for s in range(evener_mus.shape[0]):
                grid2.append([teff,logg,feh,filt,even_mus[s],interp_I[s],mu_fac,r_fac])

        bar.update()

In [ ]:
df = pd.DataFrame(data=grid1,columns=columns)
df2 = pd.DataFrame(data=grid2,columns=columns)
df.to_csv('phoenix_intensity_table.csv')
df2.to_csv('phoenix_intensity_table_resampled.csv')